In [ ]:
# This notebook is all about really crunnching when we have an image what we would 'like' to be
# called calibrated and not. Basically, what averages will we be happy with, and which will we not.
from ese.experiment.metrics.calibration.pixelwise import TL_ECE
import torch
import matplotlib.pyplot as plt

## Example 1: Bad half-boundary.

In [ ]:
# let's make the ground truth, which is a 10x10 tensor with a white square in the middle
# we will slice off the right side of the square to make our contradiction.
ground_truth1 = torch.zeros(10, 10)
ground_truth1[3:7, 3:7] = 1
ground_truth1[3:7, 6] = 0
ground_truth1[3, 4:7] = 0

plt.imshow(ground_truth1, cmap='gray')  
plt.title('Ground Truth 1')

In [ ]:
# Let's now make some predictions that we want to break the ECE metric.
# The way we do this is by making predictions in the same bin have wildly different accuracies.
# First we make a confidence map where the boundary of the square is 0.6 and the middle is 0.1
# and the rest if 0
fore_conf_map1= torch.zeros(10, 10)
fore_conf_map1[3:7, 3:7] = 0.5
fore_conf_map1[4:6, 4:6] = 1.0
back_conf_map1 = 1 - fore_conf_map1
chann_conf_map1 = torch.stack([back_conf_map1, fore_conf_map1], dim=0)
# Get the argmax
conf_map1 = torch.max(chann_conf_map1, dim=0).values

plt.imshow(fore_conf_map1, vmin=0, vmax=1, cmap='gray')
plt.colorbar(label="Confidence")
plt.title("Confidence Map 1")

In [ ]:
# Now we adversarily make our pred predict exactly half of the 0.5 area correctly (the top left) and the other half incorrectly (the bottom right).
pred_map1 = torch.argmax(chann_conf_map1, dim=0)

plt.imshow(pred_map1, cmap='gray')
plt.title("Prediction Map 1")

In [ ]:
# Paradoxically, this will have an ECE of 0!
cal_err = TL_ECE(
    num_bins=10,
    pred_map=pred_map1,
    conf_map=conf_map1,
    label_map=ground_truth1
)["cal_score"]
print("ECE: ", cal_err)

### I would actually argue this is a bad adversary because this is what calibration is meant to do, half of the pixels at the border are correct and half are wrong... thus 0.5. But it IS illustrative of a different problem, that is that different parts of the image and somehow compensate for each other. For example, working still with one object, consider the following example.

## Example 2: A Boundary Case ... Literally

In [ ]:
# let's make the ground truth, which is a 10x10 tensor with a white square in the middle
# we will slice off the right side of the square to make our contradiction.
ground_truth2 = torch.zeros(10, 10)
ground_truth2[2:8, 2:8] = 1

plt.imshow(ground_truth2, cmap='gray')  
plt.title('Ground Truth 2')

In [ ]:
# Let's now make some predictions that we want to break the ECE metric.
# The way we do this is by making predictions in the same bin have wildly different accuracies.
# First we make a confidence map where the boundary of the square is 0.6 and the middle is 0.1
# and the rest if 0
fore_conf_map2 = torch.zeros(10, 10)
fore_conf_map2[1:9, 1:9] = 0.5
fore_conf_map2[3:7, 3:7] = 1 
fore_conf_map2[3, 3:7] = 0.5
fore_conf_map2[6, 3:7] = 0.5

back_conf_map2 = 1 - fore_conf_map2
chann_conf_map2 = torch.stack([back_conf_map2, fore_conf_map2], dim=0)
# Get the argmax
conf_map2 = torch.max(chann_conf_map2, dim=0).values

plt.imshow(fore_conf_map2, vmin=0, vmax=1, cmap='gray')
plt.colorbar(label="Confidence")
plt.title("Confidence Map 2")

In [ ]:
# Now we adversarily make our pred predict exactly half of the 0.5 area correctly (the top left) and the other half incorrectly (the bottom right).
pred_map2 = torch.argmax(chann_conf_map2, dim=0)
plt.imshow(pred_map2, cmap='gray')
plt.title("Prediction Map 2")

In [ ]:
# Paradoxically, this will have an ECE of 0!
cal_err = TL_ECE(
    num_bins=10,
    pred_map=pred_map2,
    conf_map=conf_map2,
    label_map=ground_truth2
)["cal_score"]
print("ECE: ", cal_err)

### Here we observe a different kind of failure case. Regardless of how we choose to decide the coinflip at the label, the border of pixels can be COMPLETELY incorrect and ECE report the model is perfectly calibrated. This would be ok, except that pixels at the boundary with low confidence are significantly more likely to be wrong in comparison to pixels which are more inwards, thus this failure case means that perfectly calibrated models CAN just ignore the boundary as long as the distribution of confidence goes slightly inwards.

## Maybe a way to show how much a problem this can be is we look at how the confidence changes as we go further from the boundary and the accuracy going from the boundary, if the accuracy line goes up BEFORE the confidence line then this problem can be present.